The cluster map (scatter plot) is a visualization of the clusters formed by the K-Means algorithm on the app reviews dataset. The purpose of the cluster map is to help you understand the distribution and relationships among the data points in a lower-dimensional space, making it easier to interpret the results of the clustering process.

In this specific case, the scatter plot displays the data points in a 2-dimensional space, which is achieved by using Incremental PCA for dimensionality reduction. Each point in the plot represents a single app review, and the color of the point indicates the cluster it belongs to, as determined by the K-Means clustering algorithm.

The cluster map allows you to visually assess the quality of the clustering, identify any patterns or groupings, and evaluate how well the clusters separate the reviews based on their content. By examining the cluster map, you can gain insights into the structure of your data and identify relationships or similarities among the reviews that may not be immediately apparent through other means of analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from plotly.offline import plot


def save_graph_html(fig, filename):

    plot_div = plot(fig, output_type='div')

    with open(filename, 'w') as f:
        f.write(plot_div)


nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Harsh
[nltk_data]     Clean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Harsh
[nltk_data]     Clean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Text preprocessing
def preprocess_text(text):
    text = text.lower()
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stopwords.words('english')]
    return ' '.join(words)

In [6]:
# review_score = int(input("Enter the review score: "))

df = pd.read_csv('../../Warehouse/Reviews/')

# Preprocess the data
df['content'] = df['content'].fillna('').apply(preprocess_text)

# Feature extraction
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
X = tfidf.fit_transform(df['content'])

FileNotFoundError: [Errno 2] No such file or directory: '../../Warehouse/Reviews/app_reviews_1.csv'

In [5]:
# save the model to disk 
filename = 'tfidf1.sav'
pickle.dump(tfidf, open(filename, 'wb'))
X = tfidf.fit_transform(df['content'])

NameError: name 'tfidf' is not defined

In [8]:
# Optimal clusters
best_n_clusters = 2
best_score = -1

for n_clusters in range(2, 11):
    print(f"Training for {n_clusters} clusters")
    clustering = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
    y_pred = clustering.fit_predict(X)
    score = silhouette_score(X, y_pred)
    if score > best_score:
        best_score = score
        best_n_clusters = n_clusters

print(f"Optimal number of clusters: {best_n_clusters}")

Training for 2 clusters


c:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Training for 3 clusters


c:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Training for 4 clusters


...

Training for 9 clusters


c:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Training for 10 clusters


c:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Optimal number of clusters: 10


In [9]:
best_n_clusters

10

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Dimensionality reduction
pca = PCA(n_components=3, random_state=42)
X_pca = pca.fit_transform(X.toarray())  # convert sparse matrix to dense

# Clustering algorithm (now 6 clusters instead of 2)
clustering = MiniBatchKMeans(n_clusters=6, random_state=42)
y_pred = clustering.fit_predict(X)

# Save the model to a pickle file
pickle.dump(clustering, open('minibatch_kmeans.pkl', 'wb'))

# Sample a subset of your data for visualization
# You may adjust the fraction as per your need
df_sample = df.sample(frac=0.1, random_state=42)
X_sample = X_pca[df_sample.index]
y_sample = y_pred[df_sample.index]

# Visualize the clusters in 3D
fig = px.scatter_3d(x=X_sample[:, 0], y=X_sample[:, 1], z=X_sample[:, 2], 
                    color=y_sample, title="Clusters Visualization",
                    hover_data=[df_sample['content'].tolist()])
save_graph_html(fig, 'clusters3D.html')
fig.show()

NameError: name 'X' is not defined

In [13]:
# Analyze the clusters
df['cluster'] = y_pred
for i in range(best_n_clusters):
    print(f"Cluster {i}:")
    print(df[df['cluster'] == i]['content'].value_counts().head(10))

Cluster 0:
payment issue                 16
payment problem               13
payment                        7
payment received               7
upi payment                    6
payment processing             6
payment failed                 6
slow payment                   5
payment slow                   4
payment processing problem     3
Name: content, dtype: int64
Cluster 1:
useful                                                                                                                                                                      12
getting otp                                                                                                                                                                  6
useful app                                                                                                                                                                   5
app useful                                                                                     